In [ ]:
import chess
import chess.svg
from IPython.display import SVG, display
import math

In [ ]:
pawn = 100
knight = 300
bishop = 300
rook = 500
queen = 900
king = 2500
mobilityWeight = 10
board = chess.Board()

In [ ]:
board

In [ ]:
def evaluate(voard, turn):
    mobility1 = len(list(board.legal_moves))
    
    board.push(chess.Move.null())
    mobility2 = len(list(board.legal_moves))
    board.pop()
    mobility = mobilityWeight * (mobility1 - mobility2)
    if (board.turn != turn):
        mobility = -mobility

    if not turn:
        return evaluateWhite(board) - evaluateBlack(board) + mobility
    return evaluateBlack(board) - evaluateWhite(board) + mobility


def evaluateBlack(board):
    blackpawns = len(board.pieces(1,0))
    blackknights = len(board.pieces(2,0))
    blackbishops = len(board.pieces(3,0))
    blackrooks = len(board.pieces(4,0))
    blackqueens = len(board.pieces(5,0))
    blackking = len(board.pieces(6,0))
    blackmaterial = pawn*blackpawns + knight*blackknights + bishop*blackbishops + \
                    rook*blackrooks + queen*blackqueens +king*blackking
    
    return blackmaterial

def evaluateWhite(board):
    whitepawns = len(board.pieces(1,1))
    whiteknights = len(board.pieces(2,1))
    whitebishops = len(board.pieces(3,1))
    whiterooks = len(board.pieces(4,1))
    whitequeens = len(board.pieces(5,1))
    whiteking = len(board.pieces(6,1))
    whitematerial = pawn*whitepawns + knight*whiteknights + bishop*whitebishops + \
                    rook*whiterooks + queen*whitequeens +king*whiteking

    return whitematerial

In [ ]:
def getListOfMoves(board, otherSide=False):
    moves = None
    if otherSide:
        board.push(chess.Move.null())
        moves = list(map(str, list(board.legal_moves)))
        board.pop()
    else:
        moves = list(map(str, list(board.legal_moves)))
    return moves
    

In [ ]:
def useNegaMax(board, depth):
    if depth == 0:
        return evaluate(board, board.turn)
    maxValue = -math.inf
    bestMove = None     
    for move in getListOfMoves(board):
        boardCopy = board.copy()
        boardCopy.push_uci(move)
        value = -useNegaMax(boardCopy, depth)
        if value > maxValue:
            maxValue = value
            bestMove = move
    return bestMove

In [ ]:
def useNegaScout(board, depth, alpha = -math.inf, beta = math.inf):
    if depth == 0:
        return evaluate(board, board.turn)
    minimum = -math.inf
    high = beta
    for move in getListOfMoves(board):
        boardCopy = board.copy()
        boardCopy.push_uci(move)
        value = -useNegaScout(boardCopy, depth - 1, -high, -max(alpha, minimum))
        if value > minimum:
            if (high == beta or depth < 3 or value >= beta):
                minimum = value
            else:
                minimum = -useNegaScout(boardCopy, depth - 1, -beta, -value)
        if minimum >= beta:
            return minimum
        high = max(alpha, minimum) + 1
    return minimum
                

In [ ]:
def usePVS(board, depth, alpha=-math.inf, beta=math.inf):
    if depth == 0:
        return [None, evaluate(board, board.turn)]
    
    minimum = -math.inf
    high = beta
    for move in getListOfMoves(board):
        boardCopy = board.copy()
        boardCopy.push_uci(move)
        value = -usePVS(boardCopy, depth - 1, -high, -max(alpha, minimum))[1]
        if value > minimum:
            minimum = -usePVS(boardCopy, depth - 1, -beta, -alpha)[1]
        if minimum >= beta:
            return [move, minimum]
        high = max(alpha, minimum) + 1
    return [move, minimum]
                

In [ ]:
board = chess.Board()
for i in range(10):
    move = usePVS(board, 2)[0]
    print(move)
    if not i % 2:
        print("White: ", move)
    else:
        print("Black: ", move)
    board.push_uci(move)
board